In [ ]:
import pandas as pd
from collections import defaultdict

input_file = r"C:\Users\nguye\CODE\TimeSeries\BTL\Data\2019_Data.csv"
chunksize = 500_000  # Bạn có thể tăng/giảm tùy RAM

# === BƯỚC 1: Đếm số lượng hành vi mỗi user trong THÁNG 10 ===
user_event_counts = defaultdict(int)

print("🔄 Bắt đầu đếm hành vi user tháng 10...")

for chunk in pd.read_csv(input_file, chunksize=chunksize):
    # Chuyển event_time sang datetime
    chunk['event_time'] = pd.to_datetime(chunk['event_time'], errors='coerce')
    chunk = chunk.dropna(subset=['event_time'])  # loại bỏ dòng lỗi thời gian

    # Tạo cột month
    chunk['month'] = chunk['event_time'].dt.month

    # Chỉ lấy dữ liệu tháng 10
    oct_chunk = chunk[chunk['month'] == 10]

    # Đếm hành vi từng user
    counts = oct_chunk['user_id'].value_counts()
    for uid, cnt in counts.items():
        user_event_counts[uid] += cnt


print(f"✅ Đã đếm xong hành vi của {len(user_event_counts)} user trong tháng 10.")

🔄 Bắt đầu đếm hành vi user tháng 10...
✅ Đã đếm xong hành vi của 3022290 user trong tháng 10.


In [3]:
# === BƯỚC 2: Gán nhóm A/B/C/D dựa trên số lượng hành vi ===
def assign_group(n):
    if n >= 100:
        return 'A'
    elif n >= 30:
        return 'B'
    elif n >= 5:
        return 'C'
    else:
        return 'D'

user_groups = {uid: assign_group(cnt) for uid, cnt in user_event_counts.items()}

In [4]:
# === BƯỚC 3: Lướt lại file theo chunk, gán nhóm và xuất ra 4 file riêng ===
output_files = {
    'A': open("2019_Data_GroupA.csv", "w", encoding='utf-8'),
    'B': open("2019_Data_GroupB.csv", "w", encoding='utf-8'),
    'C': open("2019_Data_GroupC.csv", "w", encoding='utf-8'),
    'D': open("2019_Data_GroupD.csv", "w", encoding='utf-8')
}

header_written = {'A': False, 'B': False, 'C': False, 'D': False}

print("📤 Đang phân chia và xuất dữ liệu theo nhóm...")

for chunk in pd.read_csv(input_file, chunksize=chunksize):
    # Giữ lại các user có trong user_groups (xuất hiện trong tháng 10)
    chunk = chunk[chunk['user_id'].isin(user_groups.keys())]

    # Gán nhóm mới
    chunk['group'] = chunk['user_id'].map(user_groups)

    # Xuất theo từng nhóm
    for group in ['A', 'B', 'C', 'D']:
        sub = chunk[chunk['group'] == group].drop(columns='group')
        if not sub.empty:
            sub.to_csv(output_files[group], index=False, header=not header_written[group])
            header_written[group] = True

print("✅ Hoàn tất xuất file chia theo nhóm.")

# Đóng file
for f in output_files.values():
    f.close()

📤 Đang phân chia và xuất dữ liệu theo nhóm...
✅ Hoàn tất xuất file chia theo nhóm.
